## Final Model - Logistic Regression codes version

We’ve included here a streamlined version of our project that highlights only the critical steps for our best-performing model, which in this case is Logistic Regression. A more detailed discussion, along with the process of comparing and selecting the top algorithm from the three candidates, is available in the second notebook.

#### Importing the libraries

In [21]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

#### Datasets introduction

In [22]:
train = pd.read_csv("insurance_train.csv")
test = pd.read_csv("insurance_test.csv")

#### Logaritmization

In [23]:
train['revenue_log'] = np.log1p(train['revenue'] - train['revenue'].min() + 1)
train['reward_log'] = np.log1p(train['reward'])
train['trip_length_log'] = np.log1p(train['trip_length'])

#### Creating a list of columns

In [24]:
numeric_cols = [
    'reward_log',
    'support_interactions',
    'customer_score',
    'person_age',
    'revenue_log',
    'trip_length_log'
]
categorical_cols = [
    'person_gender',
    'entity_type',
    'channel',
    'agent_id',
    'entity_a',
    'location',
    'product_id'
]

#### Evaluation metrics function

In [25]:
def classification_metrics(y_true, y_pred_prob, cutoff = 0.5):
    y_pred = (y_pred_prob > cutoff).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    accuracy = (cm[0,0] + cm[1,1]) / np.sum(cm)
    precision_1 = cm[1,1] / (cm[1,1] + cm[0,1])
    precision_0 = cm[0,0] / (cm[0,0] + cm[1,0])
    recall_1 = cm[1,1] / (cm[1,1] + cm[1,0])
    recall_0 = cm[0,0] / (cm[0,0] + cm[0,1])
    f1_score_1 = 2 * precision_1 * recall_1 / (precision_1 + recall_1)
    f1_score_0 = 2 * precision_0 * recall_0 / (precision_0 + recall_0)
    # averages of the metrics for both classes
    balanced_accuracy = (recall_0 + recall_1) / 2
    balanced_precision = (precision_0 + precision_1)/2
    balanced_f1 = (f1_score_0 + f1_score_1) / 2

    return {'AUROC': roc_auc, 'Accuracy': accuracy, 'Precision 1': precision_1,
            'Precision 0': precision_0, 'Recall 1': recall_1, 'Recall 0': recall_0,
            'F1 Score 1': f1_score_1, 'F1 Score 0': f1_score_0, 'Balanced Accuracy': balanced_accuracy,
            'Balanced precision': balanced_precision, 'Balanced F1': balanced_f1}

#### Splitting the dataset

In [26]:
target = 'claim_status'
X = train.drop(columns=[target])
y = train[target]
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.20,
    stratify=y,
    random_state=42
)

#### Training the model with the best parameters

In [27]:
best_params = {
    'C': 0.01,
    'penalty': 'l2',
    'solver': 'liblinear',
    'l1_ratio': None,
    'sm_k_neighbors': 3
}

scaler = StandardScaler().fit(X_train[numeric_cols])
X_train_num = scaler.transform(X_train[numeric_cols])


ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False) \
        .fit(X_train[categorical_cols])
X_train_cat = ohe.transform(X_train[categorical_cols])
X_train_proc = np.hstack([X_train_num, X_train_cat])

sm = SMOTE(k_neighbors=best_params['sm_k_neighbors'], random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train_proc, y_train)

lr_model = LogisticRegression(
    C=best_params['C'],
    penalty=best_params['penalty'],
    solver=best_params['solver'],
    l1_ratio=best_params['l1_ratio'],
    class_weight='balanced',
    max_iter=2500,
    random_state=42,
    n_jobs=-1
).fit(X_train_bal, y_train_bal)

X_test_num  = scaler.transform(X_val[numeric_cols])
X_test_cat  = ohe.transform(X_val[categorical_cols])
X_test_proc = np.hstack([X_test_num, X_test_cat])

y_pred_prob = lr_model.predict_proba(X_test_proc)[:, 1]

metrics_lg = classification_metrics(y_val, y_pred_prob, cutoff=0.5)
print("Evaluation on test set:")
for name, value in metrics_lg.items():
    print(f"{name:>17}: {value:.4f}")

c:\Users\Weronika\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1271: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


Evaluation on test set:
            AUROC: 0.8412
         Accuracy: 0.7980
      Precision 1: 0.0543
      Precision 0: 0.9959
         Recall 1: 0.7784
         Recall 0: 0.7983
       F1 Score 1: 0.1014
       F1 Score 0: 0.8862
Balanced Accuracy: 0.7883
Balanced precision: 0.5251
      Balanced F1: 0.4938


#### Predictions on test dataset

In [28]:
best_params = {
    'C':             0.01,
    'penalty':       'l2',
    'solver':        'liblinear',
    'l1_ratio':      None,
    'sm_k_neighbors':3
}

# Fit final scalers and encoders on the full training set
scaler_final = StandardScaler().fit(X[numeric_cols])
ohe_final    = OneHotEncoder(handle_unknown='ignore', sparse_output=False) \
                   .fit(X[categorical_cols])

# Preprocess all train data, SMOTE application
X_num_all    = scaler_final.transform(X[numeric_cols])
X_cat_all    = ohe_final.transform(X[categorical_cols])
X_proc_all   = np.hstack([X_num_all, X_cat_all])

X_bal, y_bal = SMOTE(k_neighbors=3, random_state=42) \
                  .fit_resample(X_proc_all, y)

# Logistic regression final train
clf_final = LogisticRegression(
    C=0.01,
    penalty='l2',
    solver='liblinear',
    l1_ratio=None,
    class_weight='balanced',
    max_iter=5000,
    random_state=42
)
clf_final.fit(X_bal, y_bal)

# Create log-transformed columns in test set 
test = test.copy()
test['revenue_log'] = np.log1p(test['revenue'] - train['revenue'].min() + 1)
test['reward_log'] = np.log1p(test['reward'])
test['trip_length_log'] = np.log1p(test['trip_length'])

X_test_num  = scaler_final.transform(test[numeric_cols])
X_test_cat  = ohe_final.transform(test[categorical_cols])
X_test_proc = np.hstack([X_test_num, X_test_cat])

# Prediction
y_test_pred = clf_final.predict(X_test_proc)

# Saving to CSV
output = pd.DataFrame({
    'id': test.index,       
    'prediction': y_test_pred
})
output.to_csv('predictions.csv', index=False)
print("Written predictions.csv with", len(output), "rows")


Written predictions.csv with 6333 rows
